# Check Number of FEL trains (single bunch) in TimePix HDF5 files

In [1]:
%pylab qt

Populating the interactive namespace from numpy and matplotlib


#### Imports and utility functions

In [2]:
from src.timepix_utils import *
import src.timepix_utils as timepix_utils

timepix_utils.file_system = 'core'

### Interval of runs

In [33]:
#run_interval = [526, 560]   # 1st 400nm scan on iodopyridine
#run_interval = [563, 607]   # 2nd 400nm scan on iodopyridine
run_interval = [610, 670]

### List number of trains in runs

In [34]:
list_number_of_trains_vs_run(run_interval)

Run 665 do not exist!
Run 666 do not exist!
Run 667 do not exist!
Run 668 do not exist!
Run 669 do not exist!
Run 670 do not exist!
RunNumber: 610 | number of trains: 5871
RunNumber: 611 | number of trains: 5928
RunNumber: 612 | number of trains: 5915
RunNumber: 613 | number of trains: 7799
RunNumber: 614 | number of trains: 5914
RunNumber: 615 | number of trains: 5881
RunNumber: 616 | number of trains: 5881
RunNumber: 617 | number of trains: 5991
RunNumber: 618 | number of trains: 6056
RunNumber: 619 | number of trains: 6666
RunNumber: 620 | number of trains: 5910
RunNumber: 621 | number of trains: 5884
RunNumber: 622 | number of trains: 5887
RunNumber: 623 | number of trains: 5918
RunNumber: 624 | number of trains: 5907
RunNumber: 625 | number of trains: 5917
RunNumber: 626 | number of trains: 5891
RunNumber: 627 | number of trains: 6552
RunNumber: 628 | number of trains: 5948
RunNumber: 629 | number of trains: 5915
RunNumber: 630 | number of trains: 6007
RunNumber: 631 | number of t

### Plot number of trains in runs

In [ ]:
plot_number_of_trains_vs_run(run_interval)

### List number of trains in delay steps

In [ ]:
list_number_of_trains_vs_delay(run_interval)

### List number of trains in delay steps

In [ ]:
plot_number_of_trains_vs_delay(run_interval)
